# Look at the data

In [ ]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

!pip install pymorphy2
import re
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from nltk.corpus import stopwords

from multiprocessing import Pool
from tqdm import tqdm

import nltk
nltk.download('stopwords')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

np.random.seed(42)
random.seed(42)

Предварительно загрузим все данные на гугл диск, чтобы работать на мощностях Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/hse-nlp-bootcamp/train_ml.csv')
df

# First EDA and Time Features Creating

Сразу видим, что в столбце с целевой переменной (`grades`) есть пропуски. Удалим просто строки с пропусками в целевой переменной (не будем ничего с ними придумывать).

Посмотрим предварительно еще, есть ли пропуски в признаках

In [ ]:
df.info()

Сразу преобразуем столбец с меткой времени к соответствующему формату. И удалим строчки с пропусками в таргете

In [ ]:
df.dtypes

In [ ]:
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y %H:%M')
df = df.dropna()
df = df.astype({'grades': 'int32'})
df

Разберемся с фичами времени

*   Сделаем фичи Год, Месяц, День
*   А также фичу "время суток" из данных по часам и минутам (утро, день, вечер, ночь)



In [ ]:
def time_day(hour):
  if 0 <= hour < 6:
    return 'night'
  elif 6 <= hour < 12:
    return 'morning'
  if 12 <= hour < 18:
    return 'afternoon'
  if 18 <= hour < 24:
    return 'evening'

In [ ]:
df['year'] = df['date'].apply(lambda item: item.year)
df['month'] = df['date'].apply(lambda item: item.month)
df['day'] = df['date'].apply(lambda item: item.day)
df['time_day'] = df['date'].apply(lambda item: time_day(item.hour))

In [ ]:
df

Посмотрим на распределение числа отзывов по банкам

In [ ]:
feeds_by_bank_count = df.bank.value_counts()
feeds_by_bank_count

In [ ]:
plt.figure(figsize=(40,20))
sns.histplot(data=df.sort_values(by='bank'), 
             x=df.bank, )
plt.xticks(rotation=45)
plt.show()

Глянем на распределение клиентских оценок

In [ ]:
df.grades.value_counts().sort_index()

In [ ]:
plt.figure(figsize=(20, 10))
sns.histplot(data=df, 
             x=df.grades, )
plt.show()

# Также нас интересуют длины отзывов

In [ ]:
df['sym_len'] = df.feeds.apply(len)
df['word_len'] = df.feeds.apply(lambda x: len(x.split()))

In [ ]:
plt.figure(figsize=(15, 10))
sns.histplot(data=df.sym_len)
plt.show()

In [ ]:
plt.figure(figsize=(15, 10))
sns.histplot(data=df.word_len)
plt.show()

Обратите внимание, у распределений очень длинные хвосты

Обратили, поэтому заведем логарифмированными эти фичи:

In [ ]:
df['sym_len'] = np.log(df['sym_len'])
df['word_len'] = np.log(df['word_len'])

In [ ]:
df['sym_len'].hist()

In [ ]:
df['word_len'].hist()

# Lemmas creating

Подготавливаем тексты отзывов

In [ ]:
m = MorphAnalyzer()
regex = re.compile("[А-Яа-яA-z]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]


mystopwords = stopwords.words('russian') 
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 3]

def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    
    return ' '.join(remove_stopwords(lemmas))

In [ ]:
from multiprocessing import Pool as PoolSklearn
with PoolSklearn(4) as p:
    lemmas = list(tqdm(p.imap(clean_text, df['feeds']), total=len(df)))
    
df['lemmas'] = lemmas
df.sample(5)

In [ ]:
df['grades'].hist()

In [ ]:
df['bank'].value_counts()

In [ ]:
df[df['bank']=='sberbank']['grades'].hist()

In [ ]:
import seaborn as sns
sns.pairplot(df, hue='grades')

# Pipeline. Catboost

Будем использовать catboost. В catboost встроен хороший обработчик текстовых фичей. Он проводит при желании все операции на GPU, что делает обучение очень быстрым.

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier
from catboost import Pool

#подбор параметров не включаю в ноутбук, чтобы не засорять. Оставлю только лучшую модель

def fit_model(train_pool, validation_pool, **kwargs):
    model = CatBoostClassifier(
        iterations=25000,
        learning_rate=0.009,
        eval_metric='MultiClass',
        #early_stopping_rounds=30,
        use_best_model= True,
        task_type='GPU',
        **kwargs
    )

    return model.fit(
        train_pool,
        eval_set=validation_pool,
        verbose=100,
    )

In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split as tts
df.reset_index(drop=True, inplace=True)

df_train_val = df[['bank', 'feeds', 'lemmas', 'year', 'month', 'day', 'time_day', 'sym_len', 'word_len']]
y_train_val = df['grades']
X_train, X_val, y_train, y_val = tts(df_train_val, y_train_val, shuffle=True, stratify=y_train_val, train_size=0.8)  

In [ ]:
train_pool = Pool(
    X_train, y_train, 
    cat_features=['bank', 'time_day', 'year', 'month', 'day'],
    text_features=['lemmas', 'feeds'],
)

validation_pool = Pool(
    X_val, y_val, 
    cat_features=['bank', 'time_day', 'year', 'month', 'day'],
    text_features=['lemmas', 'feeds'],
)

print('Train dataset shape: {}\n'.format(train_pool.shape))

model = fit_model(train_pool, validation_pool)

Построим график важности признаков при обучении catboost:

In [ ]:
def plot_feature_importance(importance,names,model_type):
    
    #Create arrays from feature importance and feature names
    feature_importance = np.array(importance)
    feature_names = np.array(names)
    
    #Create a DataFrame using a Dictionary
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)
    
    #Sort the DataFrame in order decreasing feature importance
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    
    #Define size of bar plot
    plt.figure(figsize=(10,8))
    #Plot Searborn bar chart
    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
    #Add chart labels
    plt.title(model_type + 'FEATURE IMPORTANCE')
    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')
#plot the catboost result
plot_feature_importance(model.get_feature_importance(),X_train.columns,'CATBOOST ')

Очевидно, фичи с текстом оказались сильно важны для обучения, добавление еще текстовых фичей, сгенерированных из этих, думаю, может помочь улучшить качество еще сильнее. Либо какие-то статистики со знаками препинания.

Посчитаем целевую метрику на валидации:

In [ ]:
from sklearn.metrics import f1_score
val_preds = model.predict(validation_pool).flatten()
f1_score(val_preds, y_val, average='micro')

Посмотрим на confusion matrix результатов. В принципе всё очевидно, многочисленные классы лучше предсказались

In [ ]:
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

cm = confusion_matrix(y_val, val_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot() 

обучимся финально на почти всех имеющихся данных:

In [ ]:
from sklearn.model_selection import train_test_split as tts
df.reset_index(drop=True, inplace=True)

df_train_val = df[['bank', 'feeds', 'lemmas', 'year', 'month', 'day', 'time_day', 'sym_len', 'word_len']]
y_train_val = df['grades']
X_train, X_val, y_train, y_val = tts(df_train_val, y_train_val, shuffle=True, stratify=y_train_val, train_size=0.999)

In [ ]:
train_pool = Pool(
    X_train, y_train, 
    cat_features=['bank', 'time_day', 'year', 'month', 'day'],
    text_features=['lemmas', 'feeds'],
)

validation_pool = Pool(
    X_val, y_val, 
    cat_features=['bank', 'time_day', 'year', 'month', 'day'],
    text_features=['lemmas', 'feeds'],
)

print('Train dataset shape: {}\n'.format(train_pool.shape))

model = fit_model(train_pool, validation_pool)

# Inference

Загрузим тест. Предобработаем отзывы в нем и сделаем предсказания

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/hse-nlp-bootcamp/new_test_ml.csv', index_col=0)
test

In [ ]:
from multiprocessing import Pool as PoolSklearn
with PoolSklearn(4) as p:
    lemmas = list(tqdm(p.imap(clean_text, test['feeds']), total=len(test)))
    
test['lemmas'] = lemmas
test.sample(5)

In [ ]:
test['date'] = pd.to_datetime(test['date'], format='%d.%m.%Y %H:%M')
test['year'] = test['date'].apply(lambda item: item.year)
test['month'] = test['date'].apply(lambda item: item.month)
test['day'] = test['date'].apply(lambda item: item.day)
test['time_day'] = test['date'].apply(lambda item: time_day(item.hour))
test['sym_len'] = np.log(test.feeds.apply(len))
test['word_len'] = np.log(test.feeds.apply(lambda x: len(x.split())))

In [ ]:
test

In [ ]:
test.drop('date', axis=1, inplace=True)

In [ ]:
test_pool = Pool(
    test,
    cat_features=['bank', 'time_day', 'year', 'month', 'day'],
    text_features=['feeds', 'lemmas'],
)
pred = model.predict(test_pool)
pred.shape

In [ ]:
sol = pd.DataFrame({'inds': test.index,
                    'grades': pred.flatten()})
sol

In [ ]:
sol.grades.value_counts()

In [ ]:
sol.to_csv('new_baseline.csv', index=False)